In [1]:
import pandas as pd
import numpy as np
import re
import decimal

In [2]:
google_play_store=pd.read_csv("googleplaystore.csv")

In [3]:
google_reviews=pd.read_csv("googleplaystore_user_reviews.csv")

In [4]:
google_play_store.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


## 1. Convertire Size to Number

In [5]:
# Elimino caratteri da Size

google_play_store["Size"]=google_play_store["Size"].map(lambda x: x.rstrip("Mk+"))

In [6]:
#Converto in numero

google_play_store["Size"]=google_play_store["Size"].apply(pd.to_numeric,errors='coerce')


In [7]:
type(google_play_store["Size"][0])

numpy.float64

# 2. Convertire Installs to number

In [8]:
#elimino il + in fondo al numero

google_play_store["Installs"]=(google_play_store["Installs"].str.strip("+"))

In [9]:
#tolga la virgola 

google_play_store["Installs"]=google_play_store["Installs"].replace("," , "", regex=True)

In [10]:
#lo converto a numero

google_play_store["Installs"]=google_play_store["Installs"].apply(pd.to_numeric, errors='coerce')


In [11]:
type(google_play_store["Installs"][0])

numpy.float64

## 3 eliminate Varies with device.

In [12]:
google_play_store[google_play_store=="Varies with device"]=np.nan

#elimina su tutto il dataset Varies with device

C:\Users\Cristina\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


## 4 Convert Current Ver and Android Ver into a dotted number (e.g. 4.0.3 or 4.2)

In [13]:
#google_play_store.head()

In [14]:
google_play_store["Current Ver"] = google_play_store["Current Ver"].str.extract("([0-9]{1}([.][0-9]{1}){1,2})", expand=False)[0]

In [15]:
#print(google_play_store["Current Ver"])

In [16]:
#elimino caratteri finali
google_play_store["Android Ver"]=(google_play_store["Android Ver"].str.strip("and up"))
google_play_store["Android Ver"]=(google_play_store["Android Ver"].str.strip("W"))

In [17]:
#print(google_play_store["Android Ver"])

## 5. rimuovi i duplicati

In [18]:
google_play_store=pd.DataFrame.drop_duplicates(google_play_store)


In [19]:
google_play_store.index=range(len(google_play_store.index))

## 6. per ogni categoria conta il numero di app

In [20]:
google_play_store.groupby("Category").count()["App"]

Category
1.9                       1
ART_AND_DESIGN           65
AUTO_AND_VEHICLES        85
BEAUTY                   53
BOOKS_AND_REFERENCE     230
BUSINESS                427
COMICS                   60
COMMUNICATION           366
DATING                  196
EDUCATION               130
ENTERTAINMENT           111
EVENTS                   64
FAMILY                 1943
FINANCE                 360
FOOD_AND_DRINK          124
GAME                   1121
HEALTH_AND_FITNESS      306
HOUSE_AND_HOME           80
LIBRARIES_AND_DEMO       85
LIFESTYLE               373
MAPS_AND_NAVIGATION     137
MEDICAL                 408
NEWS_AND_MAGAZINES      264
PARENTING                60
PERSONALIZATION         388
PHOTOGRAPHY             322
PRODUCTIVITY            407
SHOPPING                224
SOCIAL                  280
SPORTS                  351
TOOLS                   843
TRAVEL_AND_LOCAL        237
VIDEO_PLAYERS           175
WEATHER                  82
Name: App, dtype: int64

## 7. For each category, compute the average rating

In [21]:
google_play_store.groupby("Category").mean()["Rating"]

Category
1.9                    19.000000
ART_AND_DESIGN          4.358065
AUTO_AND_VEHICLES       4.190411
BEAUTY                  4.278571
BOOKS_AND_REFERENCE     4.347458
BUSINESS                4.102593
COMICS                  4.155172
COMMUNICATION           4.151466
DATING                  3.971698
EDUCATION               4.375969
ENTERTAINMENT           4.136036
EVENTS                  4.435556
FAMILY                  4.191153
FINANCE                 4.127445
FOOD_AND_DRINK          4.164151
GAME                    4.281285
HEALTH_AND_FITNESS      4.261450
HOUSE_AND_HOME          4.164706
LIBRARIES_AND_DEMO      4.178462
LIFESTYLE               4.096066
MAPS_AND_NAVIGATION     4.051613
MEDICAL                 4.182450
NEWS_AND_MAGAZINES      4.128505
PARENTING               4.300000
PERSONALIZATION         4.333871
PHOTOGRAPHY             4.182895
PRODUCTIVITY            4.201796
SHOPPING                4.251485
SOCIAL                  4.254918
SPORTS                  4.225175
T

## 8.Create two dataframes: one for the genres and one bridging apps and genres. So that, for instance, the app Pixel Draw - Number Art Coloring Book appears twice in the bridging table, once for Art & Design, once for Creativity

In [22]:
dbGenres=pd.DataFrame(data=google_play_store["Genres"].unique())
dbGenres

,0
0,Art & Design
1,Art & Design;Pretend Play
2,Art & Design;Creativity
3,Art & Design;Action & Adventure
4,Auto & Vehicles
5,Beauty
6,Books & Reference
7,Business
8,Comics
9,Comics;Creativity


In [23]:
dbAppGen=pd.DataFrame()
dbAppGen['App']= (google_play_store["App"])
dbAppGen['Genres']=(google_play_store["Genres"])
dbAppGen

,App,Genres
0,Photo Editor & Candy Camera & Grid & ScrapBook,Art & Design
1,Coloring book moana,Art & Design;Pretend Play
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",Art & Design
3,Sketch - Draw & Paint,Art & Design
4,Pixel Draw - Number Art Coloring Book,Art & Design;Creativity
5,Paper flowers instructions,Art & Design
6,Smoke Effect Photo Maker - Smoke Editor,Art & Design
7,Infinite Painter,Art & Design
8,Garden Coloring Book,Art & Design
9,Kids Paint Free - Drawing Fun,Art & Design;Creativity


## 9.	For each genre, create a new column of the original dataframe. The new columns must have boolean values (True if the app has a given genre)

In [24]:
Generi=list(google_play_store["Genres"].unique())

In [25]:
for elem in Generi:
    google_play_store[elem]= elem == google_play_store["Genres"]

In [26]:
google_play_store

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,...,Books & Reference;Creativity,Books & Reference;Education,Puzzle;Education,Role Playing;Education,Role Playing;Brain Games,Strategy;Education,Racing;Pretend Play,Communication;Creativity,"February 11, 2018",Strategy;Creativity
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,10000.0,Free,0,Everyone,Art & Design,...,False,False,False,False,False,False,False,False,False,False
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000.0,Free,0,Everyone,Art & Design;Pretend Play,...,False,False,False,False,False,False,False,False,False,False
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,5000000.0,Free,0,Everyone,Art & Design,...,False,False,False,False,False,False,False,False,False,False
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,50000000.0,Free,0,Teen,Art & Design,...,False,False,False,False,False,False,False,False,False,False
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,100000.0,Free,0,Everyone,Art & Design;Creativity,...,False,False,False,False,False,False,False,False,False,False
5,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6,50000.0,Free,0,Everyone,Art & Design,...,False,False,False,False,False,False,False,False,False,False
6,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19.0,50000.0,Free,0,Everyone,Art & Design,...,False,False,False,False,False,False,False,False,False,False
7,Infinite Painter,ART_AND_DESIGN,4.1,36815,29.0,1000000.0,Free,0,Everyone,Art & Design,...,False,False,False,False,False,False,False,False,False,False
8,Garden Coloring Book,ART_AND_DESIGN,4.4,13791,33.0,1000000.0,Free,0,Everyone,Art & Design,...,False,False,False,False,False,False,False,False,False,False
9,Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121,3.1,10000.0,Free,0,Everyone,Art & Design;Creativity,...,False,False,False,False,False,False,False,False,False,False


## 10.	For each genre, compute the average rating. What is the genre with highest average?

In [27]:
avg_rating=google_play_store.groupby("Genres").mean()["Rating"]
max_avg_rating=google_play_store.groupby("Genres").mean()["Rating"].max()

print(avg_rating.head())

print("MASSIMO",max_avg_rating)

Genres
Action                          4.279370
Action;Action & Adventure       4.313333
Adventure                       4.180822
Adventure;Action & Adventure    4.423077
Adventure;Brain Games           4.600000
Name: Rating, dtype: float64
MASSIMO 19.0


## 11.	For each app, compute the approximate income, obtain as a product of number of installs and price.

In [28]:
#elimino il carattere 
google_play_store["Price"]=(google_play_store["Price"].str.strip("$"))

In [29]:
#converto in numerico price
google_play_store["Price"]=google_play_store["Price"].apply(pd.to_numeric,errors='coerce')

In [30]:
type(google_play_store["Price"][0])

numpy.float64

In [31]:
#creo una colonna per moltiplicare Price*Installs
google_play_store["Operazione"]=np.nan

In [32]:
google_play_store["Operazione"]=google_play_store["Price"]*google_play_store["Installs"]

In [33]:
google_play_store.groupby("App")["Operazione"].sum()

App
"i DT" Fútbol. Todos Somos Técnicos.                                                                                                       0.0
+Download 4 Instagram Twitter                                                                                                              0.0
- Free Comics - Comic Apps                                                                                                                 0.0
.R                                                                                                                                         0.0
/u/app                                                                                                                                     0.0
058.ba                                                                                                                                     0.0
1. FC Köln App                                                                                                                            

## 12For each app, compute its minimum and maximum Sentiment_polarity

In [34]:
google_reviews.groupby(google_reviews["App"]).min()["Sentiment_Polarity"]

App
10 Best Foods for You                                -0.800000
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室                     -0.112500
11st                                                 -1.000000
1800 Contacts - Lens Store                           -0.300000
1LINE – One Line with One Touch                      -0.825000
2018Emoji Keyboard 😂 Emoticons Lite -sticker&gif     -0.800000
21-Day Meditation Experience                         -0.265625
2Date Dating App, Love and matching                  -0.645833
2GIS: directory & navigator                          -0.375000
2RedBeans                                            -0.800000
2ndLine - Second Phone Number                        -0.781250
30 Day Fitness Challenge - Workout at Home           -0.500000
365Scores - Live Scores                               0.000000
3D Blue Glass Water Keyboard Theme                         NaN
3D Color Pixel by Number - Sandbox Art Coloring            NaN
3D Live Neon Weed Launcher                         

In [35]:
google_reviews.groupby(google_reviews["App"]).max()["Sentiment_Polarity"]

App
10 Best Foods for You                                 1.000000
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室                      0.910000
11st                                                  1.000000
1800 Contacts - Lens Store                            0.838542
1LINE – One Line with One Touch                       1.000000
2018Emoji Keyboard 😂 Emoticons Lite -sticker&gif      1.000000
21-Day Meditation Experience                          0.587500
2Date Dating App, Love and matching                   1.000000
2GIS: directory & navigator                           1.000000
2RedBeans                                             1.000000
2ndLine - Second Phone Number                         1.000000
30 Day Fitness Challenge - Workout at Home            0.900000
365Scores - Live Scores                               1.000000
3D Blue Glass Water Keyboard Theme                         NaN
3D Color Pixel by Number - Sandbox Art Coloring            NaN
3D Live Neon Weed Launcher                         

## PARTE2 1.	For each app, compute the average number of words in its reviews 

In [36]:
#google_reviews.head()

In [37]:
#creo una colonna che conta le parole delle recensioni
google_reviews["Conta"]=google_reviews["Translated_Review"].str.split().str.len()

In [38]:
google_reviews.groupby(google_reviews["App"])["Conta"].mean()

App
10 Best Foods for You                                  6.659794
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室                       6.275000
11st                                                  12.487179
1800 Contacts - Lens Store                             9.250000
1LINE – One Line with One Touch                        3.947368
2018Emoji Keyboard 😂 Emoticons Lite -sticker&gif       5.468750
21-Day Meditation Experience                          24.112500
2Date Dating App, Love and matching                   13.157895
2GIS: directory & navigator                           18.075000
2RedBeans                                              5.307692
2ndLine - Second Phone Number                         11.250000
30 Day Fitness Challenge - Workout at Home            15.612903
365Scores - Live Scores                               10.714286
3D Blue Glass Water Keyboard Theme                          NaN
3D Color Pixel by Number - Sandbox Art Coloring             NaN
3D Live Neon Weed Launcher          

## PARTE2 2.For each app, compute its longest review 

In [39]:
google_reviews.groupby(google_reviews["App"])["Conta"].max()

App
10 Best Foods for You                                  43.0
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室                       47.0
11st                                                   76.0
1800 Contacts - Lens Store                             37.0
1LINE – One Line with One Touch                        17.0
2018Emoji Keyboard 😂 Emoticons Lite -sticker&gif       29.0
21-Day Meditation Experience                           46.0
2Date Dating App, Love and matching                    62.0
2GIS: directory & navigator                            87.0
2RedBeans                                              38.0
2ndLine - Second Phone Number                          38.0
30 Day Fitness Challenge - Workout at Home             61.0
365Scores - Live Scores                                39.0
3D Blue Glass Water Keyboard Theme                      NaN
3D Color Pixel by Number - Sandbox Art Coloring         NaN
3D Live Neon Weed Launcher                              8.0
4 in a Row                          

## PARTE2 2 3.	For each app, compute the ratio between the number of installs and the number of reviews

In [40]:
#join i due dataset
google=google_play_store.join(google_reviews.set_index("App"),on="App")

In [41]:
#google.head()

In [42]:
google.groupby('App')['Installs'].sum()/google.groupby('App')['Translated_Review'].count()

App
"i DT" Fútbol. Todos Somos Técnicos.                                                                                                          inf
+Download 4 Instagram Twitter                                                                                                                 inf
- Free Comics - Comic Apps                                                                                                                    inf
.R                                                                                                                                            inf
/u/app                                                                                                                                        inf
058.ba                                                                                                                                        inf
1. FC Köln App                                                                                                          

## 4.	Cluster the apps according to the major android version (the first two digits — e.g. for 4.0.3 the major version is 4.0)

In [43]:
google_play_store["Current Ver"]=(google_play_store["Current Ver"].str[:3]).astype(float)
#google_play_store["Current Ver"]

In [44]:
type(google_play_store["Current Ver"][0])

numpy.float64

In [45]:
#creo una colonna con la versione massima per ciascuna App
google_play_store["Max Version"]=google_play_store.groupby(["App"])["Current Ver"].transform("max")

In [46]:
#google_play_store.head()

## 5.	For each cluster, compute the average date and the last date of an update.

In [47]:
google_play_store["Last Updated"]=google_play_store["Last Updated"].apply(pd.to_datetime,errors="coerce")

In [48]:
google_play_store["Data Seconds"]=google_play_store["Last Updated"].astype(np.int64)

In [49]:
#per ogni versione massima computa la data media di undate
google_play_store.groupby("Max Version")["Data Seconds"].mean().apply(pd.to_datetime)

Max Version
0.0   2017-07-30 20:27:47.368421120
0.1   2017-10-05 00:22:51.428571392
0.2   2018-04-08 21:10:35.294117632
0.3   2018-02-08 06:00:00.000000000
0.4   2017-12-09 20:34:17.142857216
0.5   2017-03-25 16:48:00.000000000
0.6   2017-06-02 06:32:43.636363520
0.7   2018-01-29 12:00:00.000000000
0.8   2017-10-02 12:00:00.000000000
0.9   2017-10-23 10:48:00.000000000
1.0   2017-06-11 21:06:54.446952192
1.1   2017-08-22 05:22:29.999999744
1.2   2017-08-22 23:38:13.383742208
1.3   2017-06-15 15:28:28.656716544
1.4   2017-09-12 08:17:33.658536704
1.5   2017-10-28 18:51:56.417910528
1.6   2017-09-24 11:11:24.662576640
1.7   2017-11-13 13:50:00.000000000
1.8   2017-11-16 18:36:16.744186112
1.9   2017-11-30 13:55:42.857142784
2.0   2017-08-20 16:56:53.333333248
2.1   2017-12-29 10:02:41.022364160
2.2   2017-12-26 16:07:52.131147520
2.3   2017-10-10 08:13:25.594405632
2.4   2017-09-23 06:29:30.491803392
2.5   2018-03-02 01:19:07.252747264
2.6   2018-02-03 23:23:04.615384576
2.7   2017-12-11

In [51]:
#per ogni versione vediamo l'ultimo aggiornamento, last update
google_play_store.groupby("Max Version")["Data Seconds"].max().apply(pd.to_datetime)

Max Version
0.0   2018-08-07
0.1   2018-08-03
0.2   2018-08-05
0.3   2018-07-27
0.4   2018-08-01
0.5   2018-07-31
0.6   2018-07-12
0.7   2018-08-04
0.8   2018-08-03
0.9   2018-08-06
1.0   2018-08-07
1.1   2018-08-08
1.2   2018-08-06
1.3   2018-08-06
1.4   2018-08-07
1.5   2018-08-07
1.6   2018-08-07
1.7   2018-08-07
1.8   2018-08-07
1.9   2018-08-08
2.0   2018-08-07
2.1   2018-08-07
2.2   2018-08-06
2.3   2018-08-06
2.4   2018-08-07
2.5   2018-08-06
2.6   2018-08-04
2.7   2018-08-06
2.8   2018-08-06
2.9   2018-08-07
         ...    
7.0   2018-08-05
7.1   2018-08-04
7.2   2018-08-04
7.3   2018-08-08
7.4   2018-08-06
7.5   2018-08-04
7.6   2018-08-01
7.7   2018-08-05
7.8   2018-08-01
7.9   2018-07-24
8.0   2018-08-07
8.1   2018-08-06
8.2   2018-08-07
8.3   2018-08-06
8.4   2018-08-03
8.5   2018-08-04
8.6   2018-08-02
8.7   2018-08-01
8.8   2018-08-01
8.9   2018-08-02
9.0   2018-08-02
9.1   2018-08-06
9.2   2018-08-04
9.3   2018-08-06
9.4   2018-08-06
9.5   2018-08-02
9.6   2018-08-01
9.

## 6.	Excluding the free apps, what is the content rating with highest average price?

In [52]:
group=google_play_store.groupby(["Content Rating"])["Price"].mean()
group[group>0.]

Content Rating
Everyone        1.165225
Everyone 10+    0.389098
Mature 17+      0.237383
Teen            0.572670
Name: Price, dtype: float64